In [15]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def gen_serve_dist_map(file_path, player):
    events = pd.read_csv(file_path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    
    filtered_events = events[
        (events['shotInRally'] == 1) &
        (events['serverName'] == player) &
        ((events['firstServeIn'] == 1.0) | (events['secondServeIn'] == 1.0))
    ]
    
    valid_placements = ['Wide', 'T', 'Body']
    filtered_events = filtered_events[filtered_events['serveInPlacement'].isin(valid_placements)]
    filtered_events['Zone'] =  filtered_events['side'] + " " + filtered_events['serveInPlacement']

    serve_counts = filtered_events.groupby('Zone').size()
    
    won_counts = filtered_events[filtered_events['pointWonBy'] == player].groupby('Zone').size()
    won_counts = won_counts.reindex(serve_counts.index, fill_value=0)
    
    serve_dist = pd.DataFrame({
        "Zone": serve_counts.index,
        "Win Proportion": won_counts.astype(str) + '/' + serve_counts.astype(str)
    }).reset_index(drop=True)

    serve_dist['Server'] = [player, "", "", "", "", ""]
    
    serve_dist_map_json = serve_dist.to_json(orient='records')
    with open('serve_dist_map.json', 'w') as f:
        f.write(serve_dist_map_json)

    print(filtered_events.loc[filtered_events['Zone'] == 'Deuce Wide', 'pointWonBy'])


player = "Spencer Johnson"
path = "../../Match CSVs/Shot_Visuals_SpencerJohnson_OscarPintoSansano.csv"
gen_serve_dist_map(path, player)

475         Spencer Johnson
714     Oscar Pinto Sansano
765         Spencer Johnson
815         Spencer Johnson
892         Spencer Johnson
967         Spencer Johnson
1058        Spencer Johnson
1112        Spencer Johnson
1275        Spencer Johnson
Name: pointWonBy, dtype: object
